In [ ]:
%load_ext autoreload
%autoreload 2

import uproot
import awkward as ak

import matplotlib.pylab as plt
import numpy as np

import time

from hist import Hist

import babar_analysis_tools as bat

import pandas as pd

from analysis_variables import *

import os

# Open the data file or files

In [ ]:
#####################################################################
# Where are we running this?
#####################################################################
## Bellis computer
topdir= "/home/bellis/babar_data/bnv_plambda"

## My laptop
#topdir= "/Users/josieswann/BaBar_analyses/BNV_pLambda/"
#####################################################################


#####################################################################
# Get the BNV data
#####################################################################
data, data_collision = bat.load_datasets(topdir=topdir, subset='Run1')

#####################################################################
# Get the BNC data
#####################################################################
#topdir= "/home/bellis/babar_data/bnv_plambda_bnc"
#data, data_collision = bat.load_datasets(topdir=topdir, BNC=True, subset='all')
#data, data_collision = bat.load_datasets(topdir=topdir, BNC=True, subset='Run1')

# Plots and tables for signal and blinding area

In [ ]:
from analysis_variables import *
region_definitions

In [ ]:
def table_from_df(df):
    output = df.to_latex(index=False,
                  float_format="{:.4f}".format,
    )  # converts dataframe into latex readable text
    full_table = "\\begin{table}\n" # initializes the table before the beginning of the tabular 
    full_table += "\\caption{This could be the caption}\n" 
    full_table += output #includes the converted dataframe in the table
    full_table += "\\end{table}" # ends the table, same purpose as begin{table} 
    return full_table #make sure to return the print() of the full_table, otherwise it'll be one big string that latex can't handle

In [ ]:
rd = region_definitions

dict_temp = {}
dict_temp['Region'] = ['Fitting', 'Blinding']
dict_temp['MES low'] = [rd['fitting MES'][0], rd['signal MES'][0]]
dict_temp['MES high'] = [rd['fitting MES'][1], rd['signal MES'][1]]

dict_temp['DeltaE low'] = [rd['fitting DeltaE'][0], rd['signal DeltaE'][0]]
dict_temp['DeltaE high'] = [rd['fitting DeltaE'][1], rd['signal DeltaE'][1]]


dftmp = pd.DataFrame.from_dict(dict_temp)

dftmp

In [ ]:
#table = table_from_df(dftmp)

header = ['Region', '$MES$ low', '$MES$ high', '$\Delta E$ low', '$\Delta E$ high']

output = dftmp.to_latex(index=False, header=header, 
              float_format="{:.2f}".format,
)  # converts dataframe into latex readable text

full_table = "\\begin{table}\n" # initializes the table before the beginning of the tabular 
full_table += "\\centering"
full_table += "\\caption{Definition of the fitting region and blinding region for this analysis.\\label{tab:def_regions}}\n" 
full_table += output #includes the converted dataframe in the table
full_table += "\\end{table}" # ends the table, same purpose as begin{table} 
#return full_table #make sure to return the print() of the full_table, otherwise it'll be one big string that latex can't handle

table = full_table


print(table)

outfilename = 'tables/table_def_regions.tex'
outfile = open(outfilename, 'w')
outfile.write(table)
outfile.close()


# Tables of dataset statistics

Make LaTeX tables for the number of entries in each file.

In [ ]:
output = bat.create_table_of_data_skims_statistics()

print(output)
print()

# Write it out
current_dir= os.getcwd()
print(f"Writing to {current_dir}")
directory = "tables"
path= os.path.join(current_dir,directory)
if os.path.isdir(path)== False:
    os.mkdir(path)

print(f"Writing to {directory}")

outfilename = current_dir+"/"+directory+"/table_data_skim_statistics.tex"
outfile = open(outfilename,'w+')
outfile.write(output)
outfile.close()